In [5]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory


In [6]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

from typing import Dict
from time import sleep

from tqdm import tqdm

def style_string_to_dict(style: str) -> Dict[str, str]:
  attributes = style.strip(";").split("; ")
  style_dict = {}
  for line in attributes:
    [key, value] = line.split(": ")
    style_dict[key] = value
  return style_dict

def get_jumbo_photo_from_style_string(style: str) -> str:
    style_dict = style_string_to_dict(style)
    return style_dict["background-image"].split('("')[1].split("?")[0]

def get_html_soup(url):
    print("Doing request!")
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "zoomed-image"))
    )
    datos = driver.execute_script("return document.body.innerHTML")
    print("Request executed!")
    print("Request done!")
    return BeautifulSoup(datos, "lxml")

def obtain_info_from_html_soup(html_soup):
    info = []
    code = (html_soup.find("span", class_="product-code"))
    try:
      code = code.string.split(" ")[1]
      info.append(code)
    except:
      info.append(None)
    
    name = (html_soup.find("h1", class_="product-name"))
    try:
      name = name.string
      info.append(name)
    except:
      info.append(None)


    style_photo = (html_soup.find("div", class_="zoomed-image"))
    try:
      photo = get_jumbo_photo_from_style_string(style_photo["style"])
      info.append(photo)
    except:
      info.append(None)

    price = (html_soup.find("span", class_="price-best"))
    if not price:
      price = (html_soup.find("span", class_="product-sigle-price-wrapper"))
    try:
      price = price.string.strip("$")
      info.append(price)
    except:
      info.append(None)

    price_x_kg = html_soup.find("span", class_="product-price-ppum")
    if not price_x_kg:
      price_x_kg = html_soup.find("div", class_="product-price-ppum")
    try:
      print(price_x_kg)
      price_x_kg = price_x_kg.string
      print(price_x_kg)
      price_x_kg = price_x_kg.split("$")[1].strip(")").strip(" x kg")
      info.append(price_x_kg)
    except:
      info.append(None)

    return info

def get_info_from_url(url, retries=0):
    if (retries > 5):
      return []
    # try:
    html_soup = get_html_soup(url)
    info = obtain_info_from_html_soup(html_soup)
    return info
    # except:
    #   print("Something went wrong, retrying")
    #   get_info_from_url(url, retries= retries + 1)


def search_tomates_jumbo(tomates):
    lista_columnas = ["Code", "Name", "Photo", "Price", "Price x kg"]
    tomates_filtrados = []
    for index in tqdm(range(len(tomates))):
        info = get_info_from_url(tomates[index])
        tomates_filtrados.append(info)
    print(tomates_filtrados)
    df = pd.DataFrame(tomates_filtrados, columns=lista_columnas)
    return df




ModuleNotFoundError: No module named 'bs4'

In [ ]:
tomates_jumbo = [
    "https://www.jumbo.cl/tomate-larga-vida-granel/p",
    "https://www.jumbo.cl/tomate-malla-1-kg/p",
    "https://www.jumbo.cl/tomate-salad-granel/p",
    "https://www.jumbo.cl/tomate-jumbo-granel-beef-2/p"
]

df = search_tomates_jumbo(tomates_jumbo)


  0%|          | 0/4 [00:00<?, ?it/s]

Doing request!


 25%|██▌       | 1/4 [00:02<00:07,  2.34s/it]

Request executed!
Request done!
<span class="product-price-ppum">($1.890 x kg)</span>
($1.890 x kg)
Doing request!


 50%|█████     | 2/4 [00:04<00:03,  1.98s/it]

Request executed!
Request done!
<span class="product-price-ppum">($1.890 x kg)</span>
($1.890 x kg)
Doing request!
Request executed!
Request done!


 75%|███████▌  | 3/4 [00:06<00:02,  2.21s/it]

<div class="product-price-ppum">$1.950 x kg</div>
$1.950 x kg
Doing request!


100%|██████████| 4/4 [00:08<00:00,  2.09s/it]

Request executed!
Request done!
<div class="product-price-ppum">$2.190 x kg</div>
$2.190 x kg
[['261275-KG', 'Tomate granel', 'https://jumbo.vtexassets.com/arquivos/ids/416124-750-750', '662', '1.890'], ['261296', 'Tomate larga vida malla 1 kg', 'https://jumbo.vtexassets.com/arquivos/ids/453672-750-750', '1.890', '1.890'], ['899541-KG', 'Tomate salad granel', 'https://jumbo.vtexassets.com/arquivos/ids/355918-750-750', '690', '1.950'], ['1520953-KG', 'Tomate beef granel', 'https://jumbo.vtexassets.com/arquivos/ids/164301-750-750', '1.095', '2.190']]


In [ ]:
display(df)

,Code,Name,Photo,Price,Price x kg
0,261275-KG,Tomate granel,https://jumbo.vtexassets.com/arquivos/ids/4161...,662,1.890
1,261296,Tomate larga vida malla 1 kg,https://jumbo.vtexassets.com/arquivos/ids/4536...,1.890,1.890
2,899541-KG,Tomate salad granel,https://jumbo.vtexassets.com/arquivos/ids/3559...,690,1.950
3,1520953-KG,Tomate beef granel,https://jumbo.vtexassets.com/arquivos/ids/1643...,1.095,2.190
